In [1]:
import os
import torch
from utils import get_vocabs_dep_parser

In [2]:
data_dir = 'data_new'
path_train = os.path.join(data_dir, 'train.labeled')
path_test = os.path.join(data_dir, 'test.labeled')
# path_train = os.path.join(data_dir, 'train_short.labeled')
# path_test = os.path.join(data_dir, 'test_short.labeled')

# get only train vocabs to know which words are unknown in test
paths_list_train = [path_train]
word_dict_train, pos_dict_train = get_vocabs_dep_parser(paths_list_train)

paths_list_all = [path_train, path_test]
word_dict_all, pos_dict_all = get_vocabs_dep_parser(paths_list_all)

In [3]:
from data_handling import DepDataset
from torch.utils.data.dataloader import DataLoader
print("path_train -", path_train)
print("path_test -", path_test)
BATCH_SIZE = 1

train = DepDataset(word_dict_all, pos_dict_all, data_dir, 'train', padding=False)
train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
test = DepDataset(word_dict_all, pos_dict_all, data_dir, 'test', padding=False, train_word_dict=word_dict_train)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

# train = DepDataset(word_dict, pos_dict, data_dir, 'train_short', padding=False)
# train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
# test = DepDataset(word_dict, pos_dict, data_dir, 'test_short', padding=False)
# test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

path_train - data_new\train.labeled
path_test - data_new\test.labeled
idx_pos_mappings - [0, 1, 2, 3, '#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
pos_idx_mappings - {'<pad>': 0, '<unk>': 1, '<root>': 2, '<root_pos>': 3, '#': 4, '$': 5, "''": 6, '(': 7, ')': 8, ',': 9, '.': 10, ':': 11, 'CC': 12, 'CD': 13, 'DT': 14, 'EX': 15, 'FW': 16, 'IN': 17, 'JJ': 18, 'JJR': 19, 'JJS': 20, 'LS': 21, 'MD': 22, 'NN': 23, 'NNP': 24, 'NNPS': 25, 'NNS': 26, 'PDT': 27, 'POS': 28, 'PRP': 29, 'PRP$': 30, 'RB': 31, 'RBR': 32, 'RBS': 33, 'RP': 34, 'SYM': 35, 'TO': 36, 'UH': 37, 'VB': 38, 'VBD': 39, 'VBG': 40, 'VBN': 41, 'VBP': 42, 'VBZ': 43, 'WDT': 44, 'WP': 45, 'WP$': 46, 'WRB': 47, '``': 48}
idx_pos_mappings - [0, 1, 2, 3, '#', '$', "''", '(', ')', ',', '.', ':', 'CC', '

In [4]:
print("Number of Train Sentences ", len(train))
print("Number of Test Sentences ",len(test))

Number of Train Sentences  5000
Number of Test Sentences  1000


In [5]:
import torch
from torch import nn
import torch.optim as optim
from model import DnnSepParser
from trainer import Trainer
from loss import NllLoss
from utils import IGNORE_IDX

WORD_EMBEDDING_DIM = 100
TAG_EMBEDDING_DIM = 25
NUM_LAYERS = [2, 3, 4, 5, 6]
word_vocab_size = len(train.word_idx_mappings)
tag_vocab_size = len(train.pos_idx_mappings)
max_sentence_len = max(train.max_seq_len, test.max_seq_len)

acumulate_grad_steps = 50
len_train = len(train)
len_test = len(test)

for num_layer in NUM_LAYERS:
    model = DnnSepParser(WORD_EMBEDDING_DIM, TAG_EMBEDDING_DIM, num_layer, word_vocab_size, tag_vocab_size, max_sentence_len)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    if use_cuda:
        model.cuda()

    # TODO: use our function with ignore index
    # loss_function = nn.NLLLoss(ignore_index=IGNORE_IDX)
#     loss_function = nn.NLLLoss()
    loss_function = NllLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    acumulate_grad_steps = 50 # This is the actual batch_size, while we officially use batch_size=1
    trainer = Trainer(model, optimizer, loss_function, device)

#     torch.manual_seed(1)
    EPOCHS = 100

    trainer.train_dep_parser(EPOCHS, train_dataloader, test_dataloader, acumulate_grad_steps, len_train, len_test, early_stopping=5)

C:\Users\omer.antverg\Desktop\Courses\097215\hw\HW2_NEW\code_dir\model.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  import torch
C:\Users\omer.antverg\Desktop\Courses\097215\hw\HW2_NEW\code_dir\model.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  import torch
C:\Users\omer.antverg\Desktop\Courses\097215\hw\HW2_NEW\code_dir\model.py:1: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  import torch


KeyboardInterrupt: 